In [2]:
import pandas as pd 
import numpy as np 
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000)
from missingpy import MissForest
from sklearn.preprocessing import LabelEncoder
from impyute.imputation.cs import mice

In [3]:
train_x =  pd.read_csv('train_x.csv')
train_y = pd.read_csv('train_y.csv')
test = pd.read_csv('test_x.csv')

In [4]:
drop_indices = train_y[train_y['Label'].isnull()].index
keep_indices = [i for i in train_x.index if i not in drop_indices]

In [7]:
train = train_x
train['Label'] = train_y['Label']
train.head()

,ID,Expense,Income,Loan type,Occupation type,Age,Score1,Score2,Score3,Score4,Score5,Label
0,1,1830.943788,14767.28013,B,Y,1.0,0.016885,205.196182,22.521523,600.911200,3464.613291,0.0
1,2,1645.302546,15272.26775,B,Y,0.0,0.240375,194.266317,5.349117,600.888816,3374.921455,0.0
2,3,1555.026392,17482.49734,A,Y,0.0,0.213921,183.529871,-1.054954,598.596944,3331.304886,0.0
3,4,NaN,16257.66493,A,Y,0.0,0.303909,191.228965,6.971750,602.447203,3392.275849,0.0
4,5,1777.648916,16316.29914,B,X,1.0,NaN,224.074728,11.218489,605.947340,3438.864083,0.0


In [8]:
train['Loan type'].unique(), train['Occupation type'].unique()

(array(['B', 'A', nan], dtype=object),
 array(['Y', 'X', 'Z', nan], dtype=object))

In [9]:
for i in range(len(train)):
    cell = train['Loan type'].iloc[i]
    if cell == 'A':
        train['Loan type'].iloc[i] = 0
    elif cell == 'B':
        train['Loan type'].iloc[i] = 1

In [10]:
for i in range(len(train)):
    cell = train['Occupation type'].iloc[i]
    if cell == 'X':
        train['Occupation type'].iloc[i] = 0
    elif cell == 'Y':
        train['Occupation type'].iloc[i] = 1
    elif cell == 'Z':
        train['Occupation type'].iloc[i] = 2

In [11]:
train.head()

,ID,Expense,Income,Loan type,Occupation type,Age,Score1,Score2,Score3,Score4,Score5,Label
0,1,1830.943788,14767.28013,1,1,1.0,0.016885,205.196182,22.521523,600.911200,3464.613291,0.0
1,2,1645.302546,15272.26775,1,1,0.0,0.240375,194.266317,5.349117,600.888816,3374.921455,0.0
2,3,1555.026392,17482.49734,0,1,0.0,0.213921,183.529871,-1.054954,598.596944,3331.304886,0.0
3,4,NaN,16257.66493,0,1,0.0,0.303909,191.228965,6.971750,602.447203,3392.275849,0.0
4,5,1777.648916,16316.29914,1,0,1.0,NaN,224.074728,11.218489,605.947340,3438.864083,0.0


In [10]:
# Converting all columns to float data type
# We only have two categorical columns (Loan type and Occupation type) which we can 
# change back to categorical type later

# for col in train.columns:
#     train[col] = train[col].astype('float')


In [12]:
train.dtypes

ID                   int64
Expense            float64
Income             float64
Loan type           object
Occupation type     object
Age                float64
Score1             float64
Score2             float64
Score3             float64
Score4             float64
Score5             float64
Label              float64
dtype: object

In [13]:
train['Age'] = train['Age'].astype('category')
train['Occupation type'] = train['Occupation type'].astype('category')
train['Loan type'] = train['Loan type'].astype('category')
train['Label'] = train['Label'].astype('category')
train.dtypes

ID                    int64
Expense             float64
Income              float64
Loan type          category
Occupation type    category
Age                category
Score1              float64
Score2              float64
Score3              float64
Score4              float64
Score5              float64
Label              category
dtype: object

In [14]:
cat_cols = [train.columns.get_loc(col) for col in train.select_dtypes(['category']).columns.tolist()]
# missForest imputation
cat_cols

[3, 4, 5, 11]

MISSING FOREST IMPUTATION

In [15]:
forest_imputer = MissForest(random_state =42)

In [16]:
X_imputed = forest_imputer.fit_transform(train.values, cat_vars = cat_cols)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


In [17]:
forestImputedTrain = pd.DataFrame(X_imputed, columns=train.columns)

In [18]:
forestImputedTrain.head()

,ID,Expense,Income,Loan type,Occupation type,Age,Score1,Score2,Score3,Score4,Score5,Label
0,1.0,1830.943788,14767.28013,1.0,1.0,1.0,0.016885,205.196182,22.521523,600.911200,3464.613291,0.0
1,2.0,1645.302546,15272.26775,1.0,1.0,0.0,0.240375,194.266317,5.349117,600.888816,3374.921455,0.0
2,3.0,1555.026392,17482.49734,0.0,1.0,0.0,0.213921,183.529871,-1.054954,598.596944,3331.304886,0.0
3,4.0,1681.231479,16257.66493,0.0,1.0,0.0,0.303909,191.228965,6.971750,602.447203,3392.275849,0.0
4,5.0,1777.648916,16316.29914,1.0,0.0,1.0,0.298502,224.074728,11.218489,605.947340,3438.864083,0.0


In [19]:
forestImputedTrain.to_csv('forestImputedTrain.csv',index=False)

In [4]:
forestImputedTrain = pd.read_csv('forestImputedTrain.csv')

In [21]:
forestImputedTrain.head()

,ID,Expense,Income,Loan type,Occupation type,Age,Score1,Score2,Score3,Score4,Score5,Label
0,1.0,1830.943788,14767.28013,1.0,1.0,1.0,0.016885,205.196182,22.521523,600.911200,3464.613291,0.0
1,2.0,1645.302546,15272.26775,1.0,1.0,0.0,0.240375,194.266317,5.349117,600.888816,3374.921455,0.0
2,3.0,1555.026392,17482.49734,0.0,1.0,0.0,0.213921,183.529871,-1.054954,598.596944,3331.304886,0.0
3,4.0,1681.231479,16257.66493,0.0,1.0,0.0,0.303909,191.228965,6.971750,602.447203,3392.275849,0.0
4,5.0,1777.648916,16316.29914,1.0,0.0,1.0,0.298502,224.074728,11.218489,605.947340,3438.864083,0.0


In [31]:
forestImputedTrain['Occupation type'].unique()

array([1., 0., 2.])

In [23]:
x_null_labels = forestImputedTrain.loc[drop_indices]
x_train = forestImputedTrain.loc[keep_indices]

In [25]:
len(x_train)

76097

In [26]:
x_train.dtypes

ID                 float64
Expense            float64
Income             float64
Loan type          float64
Occupation type    float64
Age                float64
Score1             float64
Score2             float64
Score3             float64
Score4             float64
Score5             float64
Label              float64
dtype: object

In [28]:
x_train.describe()

,ID,Expense,Income,Loan type,Occupation type,Age,Score1,Score2,Score3,Score4,Score5,Label
count,76097.000000,76097.000000,76097.000000,76097.000000,76097.000000,76097.000000,76097.000000,76097.000000,76097.000000,76097.000000,76097.000000,76097.000000
mean,39984.681722,1733.999747,15641.650335,0.468678,1.092264,0.441529,0.187605,192.075150,9.363862,600.393683,3417.774959,0.066139
std,23085.782443,133.309937,1063.583830,0.499021,0.721614,0.496573,0.123576,28.555249,8.758607,3.828819,64.407296,0.248527
min,1.000000,1126.809192,11171.703240,0.000000,0.000000,0.000000,-0.563328,40.572797,-28.885235,581.806404,3124.413430,0.000000
25%,19994.000000,1644.243130,14927.831830,0.000000,1.000000,0.000000,0.112587,173.455362,3.510262,597.887417,3374.409602,0.000000
50%,39964.000000,1736.290800,15624.161220,0.000000,1.000000,0.000000,0.189938,191.036701,8.876584,600.093495,3418.880495,0.000000
75%,59977.000000,1824.686798,16344.943180,1.000000,2.000000,1.000000,0.265250,209.702145,14.745843,602.599159,3461.590251,0.000000
max,80000.000000,2309.129903,20728.915330,1.000000,2.000000,1.000000,0.705737,338.073551,50.691479,619.623108,3685.793552,1.000000


In [30]:
forestImputedTrain.describe()

,ID,Expense,Income,Loan type,Occupation type,Age,Score1,Score2,Score3,Score4,Score5,Label
count,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000
mean,40000.500000,1733.979707,15641.268085,0.468700,1.092250,0.441512,0.187642,192.065006,9.363245,600.393473,3417.765289,0.065600
std,23094.155105,133.304955,1063.312654,0.499022,0.721142,0.496571,0.123456,28.538296,8.750089,3.824727,64.404941,0.247583
min,1.000000,1126.809192,11171.703240,0.000000,0.000000,0.000000,-0.563328,40.572797,-28.885235,581.806404,3124.413430,0.000000
25%,20000.750000,1644.256164,14928.428293,0.000000,1.000000,0.000000,0.112717,173.445950,3.514842,597.891336,3374.415899,0.000000
50%,40000.500000,1736.234929,15624.231990,0.000000,1.000000,0.000000,0.189924,191.034981,8.879374,600.094251,3418.855078,0.000000
75%,60000.250000,1824.429192,16343.719282,1.000000,2.000000,1.000000,0.265201,209.710764,14.740572,602.595431,3461.465789,0.000000
max,80000.000000,2309.129903,20728.915330,1.000000,2.000000,1.000000,0.705737,338.073551,50.691479,619.623108,3692.731924,1.000000
